In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "/Users/koide/Documents/ironhack-code/ironhack_service_account_big_query.json"

In [4]:
from google.cloud import bigquery

In [5]:
client = bigquery.Client()

## Challenge 1

### Step 1

In [6]:
query = """

SELECT titleauthor.title_id, titles.price, titles.advance, sales.qty, titleauthor.au_id, titleauthor.royaltyper, titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 as Royalty
FROM `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
JOIN
     `ironhack-data-analytics-265219.publications.titles` titles
ON
    titleauthor.title_id = titles.title_id
JOIN
     `ironhack-data-analytics-265219.publications.sales` sales
ON
    titleauthor.title_id = sales.title_id
ORDER BY 
    titles.title_id ASC
"""

In [7]:
query_job = client.query(query=query)
df = query_job.to_dataframe()
df

TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11d83b610>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

### Step 2

In [ ]:
query2 = """
SELECT summary.title_id, SUM(summary.royalty) as royalty,
FROM (
    SELECT titleauthor.title_id, titleauthor.au_id, titles.advance, titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 as royalty
    FROM `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
    JOIN `ironhack-data-analytics-265219.publications.titles` titles
    ON titleauthor.title_id = titles.title_id
    JOIN `ironhack-data-analytics-265219.publications.sales` sales
    ON titleauthor.title_id = sales.title_id
) as summary
GROUP BY summary.title_id
"""

In [ ]:
query_job2 = client.query(query=query2)
df = query_job2.to_dataframe()
df.head()

### Step 3

In [ ]:
query3 = """
SELECT summary.au_id, SUM(summary.royalty + summary.advance) as royalty
FROM (
    SELECT titleauthor.au_id, titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 as royalty
    FROM `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
    JOIN `ironhack-data-analytics-265219.publications.titles` titles
    ON titleauthor.title_id = titles.title_id
    JOIN `ironhack-data-analytics-265219.publications.sales` sales
    ON titleauthor.title_id = sales.title_id
) as summary
ORDER BY royalty DESC
LIMIT 3
"""

In [ ]:
query_job3 = client.query(query=query3)
df = query_job3.to_dataframe()
df

## Challenge 2

In [ ]:
create_table_query = """
CREATE TABLE `ironhack-data-analytics-265219.publications.most_profiting_authors` most_profiting_authors(
    au_id String, 
    profits NUMERIC
)
"""

In [ ]:
query_job4 = client.query(query=create_table_query)